In [5]:
import os
import librosa
import soundfile as sf
from pathlib import Path


In [8]:
import os
from pathlib import Path
import numpy as np
import librosa
import soundfile as sf

SR = 22050
CLIP_SEC = 3.0
CLIP_SAMPLES = int(SR * CLIP_SEC)

gtzan_root = Path(r"C:\Users\Atif\Documents\gtzan\Data\genres_original")
out_english = Path(r"C:\Users\Atif\Documents\easy_processed\english")
out_english.mkdir(parents=True, exist_ok=True)

def safe_load_audio(path: Path, target_sr: int = SR):
    """
    Returns mono float32 audio at target_sr, or raises an exception if loading fails.
    """
    # 1) Try soundfile (libsndfile)
    try:
        y, sr_native = sf.read(str(path), always_2d=False)
        # y can be shape (n,) or (n, ch)
        if isinstance(y, np.ndarray) and y.ndim == 2:
            y = y.mean(axis=1)  # stereo -> mono
        y = y.astype(np.float32, copy=False)
        if sr_native != target_sr:
            y = librosa.resample(y, orig_sr=sr_native, target_sr=target_sr)
        return y, target_sr
    except Exception as e_sf:
        # 2) Fallback to librosa (may use audioread if available)
        y, sr = librosa.load(str(path), sr=target_sr, mono=True)
        return y.astype(np.float32, copy=False), sr

def make_3s_clips(in_wav: Path, out_dir: Path, prefix: str):
    y, _ = safe_load_audio(in_wav, SR)
    n_clips = len(y) // CLIP_SAMPLES
    for i in range(n_clips):
        clip = y[i*CLIP_SAMPLES:(i+1)*CLIP_SAMPLES]
        out_path = out_dir / f"{prefix}_{i:03d}.wav"
        sf.write(str(out_path), clip, SR, subtype="PCM_16")  # standard wav format
    return n_clips

if not gtzan_root.exists():
    raise FileNotFoundError(f"GTZAN root not found: {gtzan_root}")

total_tracks = 0
total_clips = 0
bad_files = []

for genre_dir in sorted([p for p in gtzan_root.iterdir() if p.is_dir()]):
    wav_files = sorted(genre_dir.glob("*.wav"))
    if not wav_files:
        continue

    print(f"Processing genre: {genre_dir.name} | files: {len(wav_files)}")

    for wav_path in wav_files:
        total_tracks += 1
        prefix = f"{genre_dir.name}_{wav_path.stem}"
        try:
            n = make_3s_clips(wav_path, out_english, prefix)
            total_clips += n
        except Exception as e:
            bad_files.append((str(wav_path), repr(e)))

print("\nDone.")
print(f"Total tracks attempted: {total_tracks}")
print(f"Total 3s clips created: {total_clips}")
print(f"Saved to: {out_english}")

if bad_files:
    print(f"\nUnreadable files skipped: {len(bad_files)}")
    print("First 5 bad files:")
    for bf, err in bad_files[:5]:
        print(" -", bf)



Processing genre: blues | files: 100
Processing genre: classical | files: 100
Processing genre: country | files: 100
Processing genre: disco | files: 100
Processing genre: hiphop | files: 100
Processing genre: jazz | files: 100


C:\Users\Atif\AppData\Local\Temp\ipykernel_15816\1121589234.py:31: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(str(path), sr=target_sr, mono=True)


Processing genre: metal | files: 100
Processing genre: pop | files: 100
Processing genre: reggae | files: 100
Processing genre: rock | files: 100

Done.
Total tracks attempted: 1000
Total 3s clips created: 9981
Saved to: C:\Users\Atif\Documents\easy_processed\english

Unreadable files skipped: 1
First 5 bad files:
 - C:\Users\Atif\Documents\gtzan\Data\genres_original\jazz\jazz.00054.wav


In [9]:
import shutil
from pathlib import Path

src_root = Path(r"C:\Users\Atif\Documents\Banglabeats\wavs3sec")
dst_root = Path(r"C:\Users\Atif\Documents\easy_processed\bangla")
dst_root.mkdir(parents=True, exist_ok=True)

copied = 0
skipped = 0

for wav_path in src_root.rglob("*.wav"):
    # Example wav_path:
    # ...\wavs3sec\Adhunik\3\1.wav
    rel = wav_path.relative_to(src_root)
    
    genre = rel.parts[0] if len(rel.parts) >= 1 else "UnknownGenre"
    subfolder = rel.parts[1] if len(rel.parts) >= 3 else "root"  
    # (>=3 because: genre / subfolder / file.wav ; if deeper, still uses rel.parts[1])

    # Unique output name to avoid collisions (since many folders have 1.wav, 2.wav...)
    new_name = f"{genre}_{subfolder}_{wav_path.name}"
    out_path = dst_root / new_name

    # If still collides, append counter
    if out_path.exists():
        stem, suffix = out_path.stem, out_path.suffix
        k = 2
        while (dst_root / f"{stem}_{k}{suffix}").exists():
            k += 1
        out_path = dst_root / f"{stem}_{k}{suffix}"

    try:
        shutil.copy2(wav_path, out_path)
        copied += 1
    except Exception as e:
        skipped += 1
        print(f"Skipped: {wav_path} بسبب error: {e}")

print(f"Done. Copied: {copied}, Skipped: {skipped}")
print("Bangla merged folder:", dst_root)


Done. Copied: 16170, Skipped: 0
Bangla merged folder: C:\Users\Atif\Documents\easy_processed\bangla


In [10]:
import random
from pathlib import Path

random.seed(42)

eng_dir = Path(r"C:\Users\Atif\Documents\easy_processed\english")
ban_dir = Path(r"C:\Users\Atif\Documents\easy_processed\bangla")

eng_files = list(eng_dir.glob("*.wav"))
ban_files = list(ban_dir.glob("*.wav"))

print("English clips found:", len(eng_files))
print("Bangla clips found :", len(ban_files))

N_PER_LANG = 5000  # change to 1000 if you want smaller / faster

if len(eng_files) < N_PER_LANG or len(ban_files) < N_PER_LANG:
    raise ValueError(f"Not enough files. English={len(eng_files)}, Bangla={len(ban_files)}")

eng_sample = random.sample(eng_files, N_PER_LANG)
ban_sample = random.sample(ban_files, N_PER_LANG)

all_files = eng_sample + ban_sample
all_langs = (["english"] * len(eng_sample)) + (["bangla"] * len(ban_sample))

print("Selected total:", len(all_files))
print("Example:", all_files[0].name, all_langs[0])


English clips found: 9981
Bangla clips found : 16170
Selected total: 10000
Example: classical_classical.00082_006.wav english


In [13]:
import numpy as np
import pandas as pd
import librosa

SR = 22050
N_MFCC = 40
N_FFT = 2048
HOP = 512

def mfcc_mean_std(path):
    y, _ = librosa.load(str(path), sr=SR, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=SR, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP)
    # mfcc shape: (40, frames)
    mu = mfcc.mean(axis=1)   # (40,)
    sd = mfcc.std(axis=1)    # (40,)
    return np.concatenate([mu, sd], axis=0)  # (80,)

X = np.zeros((len(all_files), 2 * N_MFCC), dtype=np.float32)
bad = []

for i, p in enumerate(all_files):
    try:
        X[i] = mfcc_mean_std(p)
    except Exception as e:
        bad.append((i, str(p), repr(e)))

print("Feature matrix shape:", X.shape)
print("Bad files:", len(bad))
if bad:
    print("First bad:", bad[0])


Feature matrix shape: (10000, 80)
Bad files: 0


In [14]:
from pathlib import Path

save_dir = Path(r"C:\Users\Atif\Documents\easy_features")
save_dir.mkdir(parents=True, exist_ok=True)

if bad:
    bad_idx = set(i for i, _, _ in bad)
    keep_idx = [i for i in range(len(all_files)) if i not in bad_idx]
    
    X_clean = X[keep_idx]
    files_clean = [str(all_files[i]) for i in keep_idx]
    langs_clean = [all_langs[i] for i in keep_idx]
else:
    X_clean = X
    files_clean = [str(p) for p in all_files]
    langs_clean = all_langs

meta = pd.DataFrame({"path": files_clean, "language": langs_clean})

np.save(save_dir / "X.npy", X_clean)
meta.to_csv(save_dir / "meta.csv", index=False)

print("Saved:")
print(" -", save_dir / "X.npy", X_clean.shape)
print(" -", save_dir / "meta.csv", meta.shape)
print(meta.head())


Saved:
 - C:\Users\Atif\Documents\easy_features\X.npy (10000, 80)
 - C:\Users\Atif\Documents\easy_features\meta.csv (10000, 2)
                                                path language
0  C:\Users\Atif\Documents\easy_processed\english...  english
1  C:\Users\Atif\Documents\easy_processed\english...  english
2  C:\Users\Atif\Documents\easy_processed\english...  english
3  C:\Users\Atif\Documents\easy_processed\english...  english
4  C:\Users\Atif\Documents\easy_processed\english...  english
